In [57]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip

--2022-07-13 10:37:30--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  33.9MB/s    in 0.2s    

2022-07-13 10:37:30 (33.9 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]



In [58]:
!unzip ml-1m.zip

Archive:  ml-1m.zip
replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [59]:
import pandas
import numpy
from sklearn.model_selection import train_test_split

In [60]:
movies = pandas.read_csv('ml-1m/movies.dat', encoding = "ISO-8859-1", sep='::', header=None,
                         names=['movieID', 'title', 'genrs'])
ratings = pandas.read_csv('ml-1m/ratings.dat', encoding = "ISO-8859-1", sep='::', header=None,
                          names=['userID', 'movieID', 'rating', 'timestamp'])
users = pandas.read_csv('ml-1m/users.dat', encoding = "ISO-8859-1", sep='::', header=None,
                        names=['userID', 'sex', 'age', 'accupation','zip-code'])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [61]:
users

,userID,sex,age,accupation,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [62]:
from sklearn.preprocessing import MultiLabelBinarizer

In [63]:
movies.genrs = movies.genrs.str.split('|')

In [64]:
mlb = MultiLabelBinarizer()


In [65]:
mlb.fit(movies.genrs)

MultiLabelBinarizer()

In [66]:
mlb.transform(movies.genrs)

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [67]:
mlb.classes_ , len(mlb.classes_)

(array(['Action', 'Adventure', 'Animation', "Children's", 'Comedy',
        'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
        'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
        'Western'], dtype=object), 18)

In [68]:
movie_genres = pandas.DataFrame(mlb.transform(movies.genrs), columns=mlb.classes_)

In [69]:
movie_genres

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3880,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3881,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [70]:
movies = pandas.concat([movies, movie_genres], axis = 1)

In [71]:
movies['year'] = movies.title.str.strip().apply(lambda x: int(x[-5:-1]))

In [72]:
users

,userID,sex,age,accupation,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [73]:
pandas.get_dummies(users, columns=['sex'])

,userID,age,accupation,zip-code,sex_F,sex_M
0,1,1,10,48067,1,0
1,2,56,16,70072,0,1
2,3,25,15,55117,0,1
3,4,45,7,02460,0,1
4,5,25,20,55455,0,1
...,...,...,...,...,...,...
6035,6036,25,15,32603,1,0
6036,6037,45,1,76006,1,0
6037,6038,56,1,14706,1,0
6038,6039,45,0,01060,1,0


In [74]:
ratings

,userID,movieID,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [75]:
from datetime import datetime
ratings.timestamp = pandas.to_datetime(ratings.timestamp, unit='s')

In [76]:
ratings

,userID,movieID,rating,timestamp
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11
...,...,...,...,...
1000204,6040,1091,1,2000-04-26 02:35:41
1000205,6040,1094,5,2000-04-25 23:21:27
1000206,6040,562,5,2000-04-25 23:19:06
1000207,6040,1096,4,2000-04-26 02:20:48


In [77]:
anchor_time = datetime(2001, 1, 1)

In [78]:
train = ratings[ratings.timestamp < anchor_time]

In [79]:
test = ratings[ratings.timestamp > anchor_time]

In [80]:
URM = train.groupby('userID').rating.mean().reset_index()

In [81]:
URM.columns = ['userID', 'user_mean_rating']

In [82]:
MR = train.groupby('movieID').rating.mean().reset_index()
MR.columns = ['movieID', 'quality']

In [83]:
train = pandas.merge(train, URM, on='userID')

In [84]:
train = pandas.merge(train, MR, on='movieID')

In [91]:
test

,userID,movieID,rating,timestamp
4,1,2355,5,2001-01-06 23:38:11
10,1,595,5,2001-01-06 23:37:48
16,1,2687,3,2001-01-06 23:37:48
23,1,527,5,2001-01-06 23:36:35
25,1,48,5,2001-01-06 23:39:11
...,...,...,...,...
1000126,6040,1333,4,2001-08-10 14:35:40
1000162,6040,3362,4,2001-08-10 14:29:56
1000167,6040,161,3,2001-08-10 14:41:26
1000169,6040,2725,4,2001-08-10 14:36:20


In [86]:
xtrain = train[['user_mean_rating', 'quality']]

In [87]:
ytrain = train.rating

In [92]:
xtest = test[['userID', 'movieID']]

In [93]:
ytest = test.rating

In [88]:
from sklearn.linear_model import LinearRegression

In [90]:
lr = LinearRegression()

In [94]:
lr.fit(xtrain, ytrain)

LinearRegression()